In [0]:
SET use_cached_result = false;
SELECT  c_last_name
        ,c_first_name
        ,ca_city
        ,bought_city
        ,ss_ticket_number
        ,amt
        ,profit
FROM    (
            SELECT  ss_ticket_number
                    ,ss_customer_sk
                    ,ca_city bought_city
                    ,SUM(ss_coupon_amt) amt
                    ,SUM(ss_net_profit) profit
            FROM workspace.tpcds_sf1000_iceberg.store_sales
            ,       workspace.tpcds_sf1000_iceberg.date_dim
            ,       workspace.tpcds_sf1000_iceberg.store
            ,       workspace.tpcds_sf1000_iceberg.household_demographics
            ,       workspace.tpcds_sf1000_iceberg.customer_address
            WHERE   store_sales.ss_sold_date_sk = date_dim.d_date_sk
            AND     store_sales.ss_store_sk = store.s_store_sk
            AND     store_sales.ss_hdemo_sk = household_demographics.hd_demo_sk
            AND     store_sales.ss_addr_sk = customer_address.ca_address_sk
            AND     (
                        household_demographics.hd_dep_count = 1
                        OR      household_demographics.hd_vehicle_count = 1
            )
            AND     date_dim.d_dow IN (6,0)
            AND     date_dim.d_year IN (1998,1998 + 1,1998 + 2)
            AND     store.s_city IN ('Wildwood','Pleasant Grove','Friendship','Highland','Mount Pleasant')
            GROUP BY ss_ticket_number
                      ,ss_customer_sk
                      ,ss_addr_sk
                      ,ca_city
        ) dn
,       workspace.tpcds_sf1000_iceberg.customer
,       workspace.tpcds_sf1000_iceberg.customer_address current_addr
WHERE   ss_customer_sk = c_customer_sk
AND     customer.c_current_addr_sk = current_addr.ca_address_sk
AND     current_addr.ca_city <> bought_city
ORDER BY c_last_name,c_first_name,ca_city,bought_city,ss_ticket_number
LIMIT   100
;